In [2]:
import pandas as pd
import numpy as np

# funciones desarrolladas
from functions.cargar_data import *

In [31]:
pmigr = cargar_migrantes_internos()

In [32]:
pmigr.head()

,depto_origen,depto_destino,LOC,SECC,SEGM,VIVID,TIPO_VIVIE,HOGCOD,HOGID,PERPH02,...,PERED03_1,PERED03_2,PERED04_R,PERED05_R,PERED06_R,CODIGO_CAR,PERED08,NIVELEDU_R,loc_origen,loc_destino
0,3,1,20,1,1,5,1,102001001000501,1,1,...,11,0,2,3,4,55131,1,9,3732,1020
1,17,1,20,1,1,30,1,102001001003001,1,2,...,0,0,0,2,4,55152,1,9,17220,1020
2,18,1,20,1,1,36,1,102001001003601,1,1,...,6,0,2,2,0,33101,1,5,18220,1020
3,2,1,20,1,1,46,1,102001001004601,1,1,...,6,0,2,1,0,33101,1,5,2220,1020
4,12,1,20,1,1,93,1,102001001009301,1,2,...,4,0,2,1,0,22101,1,4,12320,1020


In [39]:
mig_loc  = pmigr.groupby(['loc_origen','loc_destino']).size().to_frame('personas_mig').reset_index()
print(mig_loc.shape)

(12323, 3)


In [34]:
poblacion = pd.read_csv('tablas/localidades_censales_2011.csv', usecols=['codloc','poblacion'])

In [38]:
print(poblacion.shape)
poblacion.head()

(615, 2)


,codloc,poblacion
0,1020,1304729
1,2220,40657
2,2521,12200
3,2522,2659
4,2621,2531


In [37]:
dd_loc = mig_loc.merge(poblacion, left_on='loc_origen', right_on='codloc')
dd_loc = dd_loc.merge(poblacion, left_on='loc_destino', right_on='codloc')

dd_loc.drop(['codloc_x', 'codloc_y'], inplace=True, axis=1)
dd_loc.rename({'poblacion_x':'pobl_origen', 'poblacion_y':'pobl_destino'}, inplace=True, axis=1)

dd_loc['cod'] = (dd_loc.loc_origen.astype(str) + dd_loc.loc_destino.astype(str)).astype(int)

print(dd_loc.shape)

dd_loc.head()

(8791, 6)


,loc_origen,loc_destino,personas_mig,pobl_origen,pobl_destino,cod
0,1020,2220,635,1304729,40657,10202220
1,3221,2220,5,71258,40657,32212220
2,3320,2220,10,19865,40657,33202220
3,3321,2220,1,20524,40657,33212220
4,3322,2220,1,25947,40657,33222220


In [26]:
# levanta distancias
dist = pd.read_csv('tablas/df_distancias_localidades.csv', usecols=['cod','distancia_m'])
dist.head()

,cod,distancia_m
0,22202521,137927
1,22202522,111408
2,22202621,106867
3,22202721,58392
4,22202724,142223


In [27]:
dd_loc = dd_loc.merge(dist, on='cod')

In [29]:
dd_loc.tail()

,loc_origen,loc_destino,personas_mig,pobl_origen,pobl_destino,cod,distancia_m
8786,18220,14965,1,54755,32,1822014965,446839
8787,18321,6940,12,12985,77,183216940,81281
8788,18321,9932,2,12985,49,183219932,244765
8789,19220,5994,1,25477,136,192205994,401841
8790,19220,18963,4,25477,26,1922018963,359313


In [30]:
dd_loc.to_csv('tablas/dd_localidades.csv', index=False)